In [10]:
import pandas as pd
%matplotlib inline 
import matplotlib.pyplot as plt

/home/vijay/tools/miniconda3/envs/deep-learner/lib/python3.6/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


## Exploration of the data

### Data dictionary 

In [6]:
df_dict = pd.read_excel('../data/LCDataDictionary.xlsx')

In [8]:
df_dict.shape

(80, 2)

In [41]:
from IPython.display import display
display(df_dict)

,LoanStatNew,Description
0,addr_state,The state provided by the borrower in the loan...
1,annual_inc,The self-reported annual income provided by th...
2,annual_inc_joint,The combined self-reported annual income provi...
3,application_type,Indicates whether the loan is an individual ap...
4,collection_recovery_fee,post charge off collection fee
5,collections_12_mths_ex_med,Number of collections in 12 months excluding m...
6,delinq_2yrs,The number of 30+ days past-due incidences of ...
7,desc,Loan description provided by the borrower
8,dti,A ratio calculated using the borrower’s total ...
9,dti_joint,A ratio calculated using the co-borrowers' tot...


### Exploring the data

In [11]:
import sqlite3

In [12]:
conn = sqlite3.connect('../data/database.sqlite')

In [16]:
cur = conn.cursor()

In [21]:
df_loan = pd.read_sql_query('select * from loan', conn)

In [22]:
df_loan.shape

(887383, 75)

In [37]:
df_loan2 = pd.read_csv('../data/loan.csv',low_memory=False)

In [40]:
df_loan2.shape

(887379, 74)

In [38]:
df_loan2.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

In [39]:
df_loan2.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df_loan['inq_last_12m'].value_counts()

 1.0     5045
 0.0     3983
 2.0     3756
 3.0     2600
 4.0     1696
-4.0     1372
 5.0      996
 6.0      670
 7.0      431
 8.0      274
 9.0      144
 10.0     115
 11.0      74
 12.0      57
 13.0      56
 14.0      27
 15.0      26
 16.0      12
 19.0       8
 17.0       8
 20.0       7
 18.0       5
 21.0       3
 25.0       2
 32.0       1
 30.0       1
 22.0       1
 26.0       1
 24.0       1
Name: inq_last_12m, dtype: int64

In [43]:
df_loan['next_pymnt_d'].unique()

array([None, 'Feb-2016', 'Jan-2016', 'Sep-2013', 'Feb-2014', 'May-2014',
       'Jun-2013', 'Mar-2012', 'Apr-2012', 'May-2013', 'Aug-2012',
       'Aug-2013', 'Jun-2012', 'Nov-2013', 'Feb-2012', 'Oct-2011',
       'Jan-2013', 'Jan-2014', 'Jul-2013', 'Jul-2015', 'Jan-2012',
       'Dec-2012', 'Jun-2011', 'Feb-2013', 'Nov-2011', 'Nov-2012',
       'Dec-2011', 'Aug-2011', 'Sep-2011', 'Apr-2011', 'Mar-2014',
       'Apr-2013', 'Mar-2011', 'Jul-2012', 'Aug-2014', 'Oct-2013',
       'Sep-2012', 'May-2012', 'Apr-2015', 'Jul-2011', 'Dec-2015',
       'Dec-2013', 'Jan-2011', 'Oct-2012', 'Nov-2014', 'Mar-2013',
       'Aug-2015', 'Feb-2015', 'May-2015', 'Jul-2014', 'Nov-2015',
       'Sep-2014', 'Oct-2015', 'May-2011', 'Feb-2011', 'Dec-2014',
       'Jun-2015', 'Apr-2014', 'Jan-2015', 'Sep-2015', 'Jun-2014',
       'Nov-2010', 'Oct-2010', 'Dec-2010', 'Mar-2015', 'Oct-2014',
       'Jul-2010', 'Sep-2010', 'May-2010', 'Aug-2010', 'Mar-2010',
       'Jun-2010', 'Apr-2010', 'Feb-2010', 'Dec-2009', '

### Check out how many members are in common between 2007-2015 and 2016 Q1

In [46]:
df_2016_q1 = pd.read_csv('../data/LoanStats_2016Q1.csv',low_memory=False)

Index(['Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)'], dtype='object')

In [ ]:
df_2016_q1[df_2016_q1['']]